# Scraping dati per creazione dataset

### Ricaviamo i dettagli tecnici di ogni versione di ogni modello di ogni marca di auto

##### import librerie

In [7]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import re

##### Creazione classe per scrape html

In [8]:
class ScrapeGenerator:
    def __init__(self, url):
        self.url = url
        self.soup = None

    async def scrape_data(self):
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
            }
            async with aiohttp.ClientSession() as session:
                async with session.get(self.url,headers=headers) as response:
                    content = await response.text()
                    soup = BeautifulSoup(content, 'html.parser')
                    self.soup = soup
                    return soup

##### Funzione per prendere i modelli in produzione di ogni marca di auto

In [9]:
async def scrape_car_brand_data(sp: BeautifulSoup):
    # lista quantita auto in produzione
    bi = [y.text for y in sp.find_all('b',{'class':'col-green2'})]
    #trova tutti gli h5 che contengono gli 'a' con href al modello
    sp = sp.find_all('h5')
    all_brands_links = []
    for x in tqdm(range(len(sp)),"scraping car brand data"):
        if bi[x] == '0':
            continue
        #aggiungi tutti i link che hanno modelli in production
        all_brands_links.append(sp[x].find('a',href=True)['href'])

    return all_brands_links

##### Funzione per ricavare i modelli di ogni marca

In [10]:
def isPresent(l: list):
    for x in l:
        if "present" in str(x).lower():
            return True
def removeBrandName(s: str, torem: str):
    if "-" in torem:
        torem = torem.replace("-"," ")
    return s.replace(torem,"").strip()

async def scrape_car_model_data(lstBrand: list):
    ret = {} #{ brand:{model:{..info}}, ...  }
    for link_brand in tqdm(lstBrand,"scraping car models"):
        nme = link_brand.split("/")[3]
        sg = ScrapeGenerator(link_brand)
        sg = await sg.scrape_data()
        imglink = sg.find_all('div',{'class':'pic'})
        imglink = imglink[0].find('img')['src']
        ret[nme] = {'imglink':imglink}
        divCars = sg.find_all('div',{'class':'carmod clearfix'})
        #prendo ogni div che contiene ogni macchina
        for car in divCars:
            isPrnt = isPresent(car.find_all('span'))
            if not isPrnt:
                continue
            #se e' ancora in production aggiungiamo il link alla lista
            mname = removeBrandName(car.h4.text, nme.upper())
            lnk = car.a['href']
            ret[nme][mname] = {'link':lnk}

    return ret

##### Funzione per ricavare ogni modello di auto

In [11]:
def getEngines(divvoneCar: BeautifulSoup, brand: str, model: str):
    boxEngines = divvoneCar.find_all('div',{'class':'mot clearfix'})
    eng = {}
    for x in boxEngines:
        for y in x.find_all('a'):
            eng[y.text] = {'link':y['href']}
    return eng
    
async def scrape_eachModel_data(dizModels: dict):
    for brand in tqdm(dizModels.keys(),f"scraping models details"):
        for model in dizModels[brand]:
            if model == "imglink": continue
            sg = ScrapeGenerator(dizModels[brand][model]['link'])
            sg = await sg.scrape_data()
            
            imgcar = sg.find('div',{'class':'col1width fl'})
            imgcar = imgcar.find('img')['src']
        
            divEngines = sg.find_all('div',{'class':'container carmodel clearfix'})
            engList = getEngines(divEngines[0],brand,model)
            dizModels[brand][model]['engines'] = engList
            dizModels[brand][model]['car_imglink'] = imgcar

    return dizModels

##### Funzione per ricavare i dettagli di ogni versione di ogni modello

In [12]:
async def scrape_eachEngine_data(dizModels: dict):
    for brand in tqdm(dizModels.keys(),f"scraping engines details"):
        for model in dizModels[brand]:
            if model == "imglink": continue
            for engine in dizModels[brand][model]['engines']:
                sg = ScrapeGenerator(dizModels[brand][model]['engines'][engine]['link'])#['link']
                sg = await sg.scrape_data()
                tableInfo = sg.find_all('div',{'class':'padcol2'})
                for y in tableInfo:
                    if "ENGINE SPECS" in y.text:
                        tableInfo = y
                        break
                try:
                    descInfo = tableInfo.find_all('td',{'class':'left'})
                    descVal = tableInfo.find_all('td',{'class':'right'})
                except:
                    print(f"{tableInfo}")

                pattern = r'\((\d+)\sHP\)'
                match = re.search(pattern, engine)
                if match:
                    dizModels[brand][model]['engines'][engine]["HP"] = match.group(1)
                else:
                    dizModels[brand][model]['engines'][engine]["HP"] = None
                
                pattern = r'\d+(?:\.\d+)?(?:cc|L)'
                match = re.search(pattern, engine)
                if match:
                    dizModels[brand][model]['engines'][engine]["Pdisplacement"] = match.group(0)
                else:
                    dizModels[brand][model]['engines'][engine]["Pdisplacement"] = None

                for x in range(len(descInfo)):
                    dizModels[brand][model]['engines'][engine][descInfo[x].text] = descVal[x].text.strip()
    with open('completo.json', 'w') as f:
        json.dump(dizModels, f)
    return dizModels

##### Funzione principale che esegue i vari passaggi

In [ ]:
async def main(url):
    sg = ScrapeGenerator(url)
    sg = await sg.scrape_data()
    lbrand = await scrape_car_brand_data(sg)
    dcarModel = await scrape_car_model_data(lbrand)
    eachModelData = await scrape_eachModel_data(dcarModel)
    scrape_eachEngine_data(eachModelData)

if __name__ == '__main__':
    url = "https://www.autoevolution.com/cars/"
    asyncio.run(main(url))

### Ricaviamo i prezzi tramite ricerca google e scraping del risultato

##### Import librerie

In [3]:
import json
import asyncio
import re
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

##### Iniziliziamo variabili globali e set delle opzioni dei webdriver di chrome

In [4]:
chrome_driver_path = "/usr/bin/google-chrome"
pathFile = "Datasets_Scraping/completo_wPrices_official.json"
pathInitFile = "Datasets_Scraping/completo.json"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.binary_location = chrome_driver_path

##### Cerchiamo nell html della pagina della ricerca di google un prezzo coerente tramite regex 

In [16]:
def convert_to_integer(number_str):
    number_str =  number_str.replace("USD","").replace("$","").replace("EUR","").replace("€","").replace("£","").replace(" ","").strip()
    pattern = r'\.\d{2}$' 
    pattern2 = r',\d{2}$'  
    if re.search(pattern, number_str):
        number_str = number_str.split(".")[0]
    elif re.search(pattern2, number_str):
        number_str = number_str.split(",")[0]
    return int(number_str.replace(',', '').replace('.', ''))

async def seleniumscrape(link):
    browser = webdriver.Chrome(options=chrome_options)
    browser.get(link)
    acc = WebDriverWait(browser, 100).until(EC.element_to_be_clickable((By.XPATH, "//button[@id='L2AGLb']")))
    acc.click()
    element = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='main']")))

    reg = r'((?:\$|€|USD)\s*\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?|\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?\$|\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?\s*(?:\$|€|USD))' #TODO check if regex works, added ?
    fnda = re.findall(reg, element.text) 
    numbers = []
    try:
        for num in fnda:
            numbers.append(convert_to_integer(num))
        max_number = max(numbers)
        if max_number > 20000000:
            numbers.remove(max_number)
            max_number = max(numbers)
        elif max_number < 2000:
            max_number = 0
    except Exception as e:
        max_number = 0
        print(f"error \n{e}\nON {link}")
    finally:
        browser.quit()
    return max_number

##### Funzione principale che esegue i vari passaggi

In [ ]:
async def main():
    with open(pathInitFile, 'r') as f:
        dcarModel = json.load(f)
        for x in tqdm(dcarModel, f"scraping prices"):
            for y in dcarModel[x]:
                if y == "imglink": continue
                lnk = f'https://www.google.com/search?q={x}+{y}+price'
                pricez = await seleniumscrape(lnk)
                dcarModel[x][y]['price'] = pricez
        with open(pathFile, 'w') as f:
            json.dump(dcarModel, f)
            
asyncio.run(main())

### Puliamo i dati del dataset

##### Import librerie

In [ ]:
import json
from tqdm import tqdm
import copy

##### Iniziliziamo variabili globali

In [ ]:
pathFile = "Datasets_Scraping/completo_wPrices_cleaner_official.json"
pathInitFile = "Datasets_Scraping/completo_wPrices_official.json"

##### Puliamo il dataset

In [ ]:
def cleanDs():
    with open(pathInitFile, 'r') as f:
        dcarModel = json.load(f)
        dcarModel_cln = copy.deepcopy(dcarModel)
        deletatiByCilindr = 0
        deletatiByEng = 0
        deletatiByPrc = 0
        for brand in tqdm(dcarModel.keys()):
            for model in dcarModel[brand].keys():
                if model == 'imglink': continue
                a = dcarModel[brand][model]
                if a['price'] == 0:
                    del dcarModel_cln[brand][model]
                    deletatiByPrc += 1
                    continue
                
                if len(a['engines'].keys()) == 0:
                    print("cancello [eng == 0]",brand, model)
                    del dcarModel_cln[brand][model]
                    deletatiByEng += 1
                    continue
                for eng in a['engines'].keys():
                    if len(a['engines'][eng].keys()) <3:
                        print("cancello [eng < 3]",brand, model, eng)
                        del dcarModel_cln[brand][model]['engines'][eng]
                        deletatiByEng += 1
                        continue
                
                    disp = a['engines'][eng].get('Displacement:')
                    pdisp = a['engines'][eng].get('Pdisplacement')
                    if disp == None and pdisp == None:
                        print("cancello [no cilindr info]",brand, model, eng)
                        del dcarModel_cln[brand][model]['engines'][eng]
                        deletatiByCilindr += 1
                    elif pdisp == None and disp:
                        numz =  "{:.1f}".format(int(disp.split()[0])/1000) 
                        dcarModel_cln[brand][model]['engines'][eng]['Pdisplacement'] = f"{numz}L"
                        
                if len(dcarModel_cln[brand][model]['engines'].keys()) == 0:
                    print("cancello [eng == 0]",brand, model)
                    del dcarModel_cln[brand][model]
                    deletatiByEng += 1
                    continue
                
        print("cancellatiByEngine", deletatiByEng)
        print("cancellatiByCilindri", deletatiByCilindr)
        print("cancellatiByPrezzo", deletatiByPrc)
                    
    with open(pathFile, 'w') as f:
        json.dump(dcarModel_cln, f)

if __name__ == '__main__':    
    cleanDs()